In [8]:
import random
import math

# Function to calculate the number of conflicts (pairs of queens attacking each other)
def calculate_conflicts(board):
    n = len(board)
    conflicts = 0
    for i in range(n):
        for j in range(i + 1, n):
            # Check if two queens are on the same row or diagonal
            if board[i] == board[j] or abs(board[i] - board[j]) == abs(i - j):
                conflicts += 1
    return conflicts

# Function to perform simulated annealing to solve the N-Queen problem
def simulated_annealing(board, max_steps=1000, initial_temp=1000, cooling_rate=0.95):
    n = len(board)
    # Initial temperature and initial conflicts
    temperature = initial_temp
    current_conflicts = calculate_conflicts(board)

    for step in range(max_steps):
        # If there are no conflicts, the solution is found
        if current_conflicts == 0:
            return board

        # Select two random positions to swap queens
        i, j = random.sample(range(n), 2)
        new_board = board[:]
        new_board[i], new_board[j] = new_board[j], new_board[i]

        # Calculate the new number of conflicts after swap
        new_conflicts = calculate_conflicts(new_board)

        # Calculate the probability of accepting the new state
        delta = new_conflicts - current_conflicts
        if delta < 0 or random.uniform(0, 1) < math.exp(-delta / temperature):
            board = new_board
            current_conflicts = new_conflicts

        # Decrease the temperature
        temperature *= cooling_rate

    # If no solution is found within max_steps
    return None

# Prompt the user for an initial board configuration
n = 8
print("Enter the initial state of the board as a list of 8 integers (0-7), each representing the row of the queen in columns 0 to 7.")
initial_state = input("Example: [0, 4, 7, 5, 2, 6, 1, 3]\nEnter your initial state: ")
initial_board = list(map(int, initial_state.strip("[]").split(",")))

# Ensure the board has 8 elements and that each element is within the range
if len(initial_board) != n or not all(0 <= x < n for x in initial_board):
    print("Invalid input. Please enter 8 integers between 0 and 7.")
else:
    # Run the simulated annealing algorithm with the user-provided initial board
    solution = simulated_annealing(initial_board)
    if solution:
        print("Solution found:", solution)
    else:
        print("No solution found within the given steps.")


Enter the initial state of the board as a list of 8 integers (0-7), each representing the row of the queen in columns 0 to 7.
Example: [0, 4, 7, 5, 2, 6, 1, 3]
Enter your initial state: 1,2,3,4,5,6,7,0
Solution found: [3, 6, 2, 7, 1, 4, 0, 5]


In [15]:
from sympy.logic.boolalg import Or, And, Not, Implies
from sympy.logic import simplify_logic
from sympy import symbols

# Function to create a Knowledge Base by taking user input
def create_knowledge_base():
    # User input for number of propositions
    num_propositions = int(input("Enter the number of propositions: "))

    # Create symbols for the propositions
    propositions = [symbols(input(f"Enter the name of proposition {i+1}: ")) for i in range(num_propositions)]

    kb = []

    # Allow the user to input knowledge base rules (as logical expressions)
    print("Enter the knowledge base as one logical expression, using 'and', 'or', 'not', 'implies' (use parentheses for grouping):")
    print("Example: (Q implies P) and (P implies not Q) and (Q or R)")
    rule = input("KB: ")

    try:
        # Parse the user input into sympy logical expressions
        rule_expr = parse_logical_expression(rule, propositions)
        kb.append(rule_expr)
    except Exception as e:
        print(f"Error parsing KB: {e}")

    return kb, propositions

# Function to parse a logical expression entered by the user
def parse_logical_expression(expr, propositions):
    # Replace user input keywords with sympy equivalents
    expr = expr.lower()
    expr = expr.replace("and", "&").replace("or", "|").replace("not", "~").replace("implies", ">>")

    # Create a dictionary of symbols
    symbol_dict = {prop.name: prop for prop in propositions}

    # Replace the proposition names with sympy symbols in the expression
    for prop in symbol_dict:
        expr = expr.replace(prop, f"symbols('{prop}')")

    # Use sympy.logic functions to parse and return the logical expression
    from sympy import sympify
    try:
        return sympify(expr)
    except Exception as e:
        raise ValueError(f"Failed to parse expression: {expr}. Error: {e}")

# Function to check if a query is entailed by the knowledge base
def entails(kb, query, symbols_list):
    # Simplify the knowledge base by combining all formulas into a single expression
    combined_kb = And(*kb)

    # Check if the query is a logical consequence of the knowledge base
    # If combined KB ∧ ¬query is unsatisfiable, then KB entails query
    result = simplify_logic(And(combined_kb, Not(query)))

    # If the result is False (unsatisfiable), the query is entailed
    if result == False:
        return True
    else:
        return False

# Example usage:

# Create knowledge base
kb, symbols_list = create_knowledge_base()

# Allo\w the user to input a query
query_str = input("Enter the query to check for entailment (use 'and', 'or', 'not', 'implies'): ")

# Parse the query entered by the user
try:
    query = parse_logical_expression(query_str, symbols_list)

    # Check if the query is entailed by the KB
    if entails(kb, query, symbols_list):
        print(f"The query {query_str} is entailed by the knowledge base.")
    else:
        print(f"The query {query_str} is not entailed by the knowledge base.")
except Exception as e:
    print(f"Error: {e}")

Enter the number of propositions: 3
Enter the name of proposition 1: P
Enter the name of proposition 2: Q
Enter the name of proposition 3: R
Enter the knowledge base as one logical expression, using 'and', 'or', 'not', 'implies' (use parentheses for grouping):
Example: (Q implies P) and (P implies not Q) and (Q or R)
KB: (Q implies P) and (P implies not Q) and (Q or R)
Enter the query to check for entailment (use 'and', 'or', 'not', 'implies'): R
The query R is entailed by the knowledge base.
